In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [2]:
@model function smoothing(n, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    c = constvar(1.0)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0

n = 10_000
k = 500
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(; data, k, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, P, options = (limit_stack_depth = k, ));

    buffer    = Vector{Marginal}(undef, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, (ms) -> copyto!(buffer, ms))
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return buffer
end

inference (generic function with 1 method)

In [5]:
@benchmark res = inference(
    data = $data,
    k = $k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = $P
)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  699.556 ms … 900.957 ms  ┊ GC (min … max): 34.30% … 49.82%
 Time  (median):     809.076 ms               ┊ GC (median):    43.07%
 Time  (mean ± σ):   794.267 ms ±  64.346 ms  ┊ GC (mean ± σ):  42.99% ±  5.84%

  ▁         ▁               ▁      █▁                         ▁  
  █▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  700 ms           Histogram: frequency by time          901 ms <

 Memory estimate: 278.13 MiB, allocs estimate: 4711260.

In [6]:
@benchmark res = inference(
    data = $data,
    k = $k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = $P
)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  622.653 ms … 872.077 ms  ┊ GC (min … max): 25.57% … 48.90%
 Time  (median):     739.828 ms               ┊ GC (median):    39.19%
 Time  (mean ± σ):   738.400 ms ±  83.173 ms  ┊ GC (mean ± σ):  38.67% ±  7.79%

  █       █                █  █   █        █                  █  
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁█▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  623 ms           Histogram: frequency by time          872 ms <

 Memory estimate: 278.13 MiB, allocs estimate: 4711260.